In [3]:
import pandas as pd
import re
import dns.resolver
from flask import Flask, render_template, request, redirect



ModuleNotFoundError: No module named 'dns'

In [ ]:
app = Flask(__name__)




In [ ]:
def clean_emails(df):
    email_columns = ['Email 1', 'Email 2', 'Email 3']
    for column in email_columns:
        df[column] = df[column].apply(lambda x: re.sub(r'\s+', '', str(x)))  # Remove whitespaces
        df[column] = df[column].apply(lambda x: re.findall(r'[\w\.-]+@[\w\.-]+', str(x)))  # Find valid email addresses
    return df

def clean_phone_numbers(df):
    phone_column = 'Main Phone'
    df[phone_column] = df[phone_column].apply(lambda x: re.sub(r'\s+', '', str(x)))  # Remove whitespaces
    df[phone_column] = df[phone_column].apply(lambda x: re.sub(r'^020|^20', '', str(x)))  # Remove '020' or '20' prefix
    return df

def validate_dns(email):
    try:
        domain = email.split('@')[1]
        dns.resolver.query(domain, 'MX')
        return True
    except (dns.resolver.NoAnswer, dns.resolver.NXDOMAIN):
        return False

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        file = request.files['file']
        filename = 'clean_' + file.filename
        if file.filename.endswith('.xlsx'):
            df = pd.read_excel(file)
        elif file.filename.endswith('.csv'):
            df = pd.read_csv(file)
        else:
            return render_template('index.html', message="Invalid file format. Only xlsx and csv files are supported.")
        
        df_cleaned = clean_emails(df)
        df_cleaned = clean_phone_numbers(df_cleaned)
        
        # Validate email addresses using DNS lookup
        email_columns = ['Email 1', 'Email 2', 'Email 3']
        for column in email_columns:
            df_cleaned[column] = df_cleaned[column].apply(lambda x: [email for email in x if validate_dns(email)])
        
        if file.filename.endswith('.xlsx'):
            df_cleaned.to_excel(filename, index=False)
        elif file.filename.endswith('.csv'):
            df_cleaned.to_csv(filename, index=False)
        else:
            return render_template('index.html', message="Something went wrong while saving the cleaned file.")
        
        return redirect('/download/' + filename)
    return render_template('index.html')



In [ ]:
@app.route('/download/<filename>', methods=['GET'])
def download_file(filename):
    return render_template('download.html', filename=filename)

if __name__ == '__main__':
    app.run()